# Data Retrieval Example

## requirements:
- instal praw using the following command `!pip install praw`. Don't use conda for this specific model.
- instal beautifulsoup using the following command `!pip install bs4`
- instal requests if you don't have in your development environment 

## References:
- https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
- https://www.reddit.com/dev/api
- https://praw.readthedocs.io/en/stable/index.html


In [29]:
import praw
import pandas as pd
import requests
import bs4

## Connecting to the reddit API using `requests`

### Steps:
- create an app on reddit to acquire the access credentials:
    - `PASSWORD` password of the reddit account used to create the app
    - `USERNAME` username of the reddit account used to create the app
    - `CLIENT_ID` acquired after creating the app
    - `SECRET_TOKEN` acquired after creating the app


In [30]:

PASSWORD = 'j=HUZ`6S8B'
USERNAME = 'CMPS287_project'
CLIENT_ID = 'd5w9jc7mmyeLEL2DG1wtxg'
SECRET_TOKEN = 'HIOuTew4HunOVSJeFT47Yi4sCkdBCA' 
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': USERNAME,
        'password': PASSWORD }

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyAPI/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
response_josn = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

# To see the response just print response_json
# print(response_josn)

In [31]:
hot = "https://oauth.reddit.com/r/python/hot"


# make a request for the trending posts in /r/Python
res = requests.get("https://oauth.reddit.com/r/python/hot",
                   headers=headers)

df = pd.DataFrame()  # initialize dataframe

# loop through each post retrieved from GET request
for post in res.json()['data']['children']:
    # append relevant data to dataframe
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    }, ignore_index=True)


In [32]:
df.head()

,downs,score,selftext,subreddit,title,ups,upvote_ratio
0,0.0,6.0,Tell /r/python what you're working on this wee...,Python,Sunday Daily Thread: What's everyone working o...,6.0,0.88
1,0.0,4.0,Have some burning questions on advanced Python...,Python,Tuesday Daily Thread: Advanced questions,4.0,0.84
2,0.0,17.0,Python just makes so much freaking sense! LOVI...,Python,Learning Python,17.0,0.76
3,0.0,79.0,,Python,I'm presenting live in 6 hours at Microsoft Re...,79.0,0.87
4,0.0,551.0,,Python,Python f-strings Are More Powerful Than You Mi...,551.0,0.93


In [33]:
# getting usernames of the hot posts
authors_contributors = "https://oauth.reddit.com/r/explainlikeimfive/hot"

# make a request for the trending posts in /r/Python
res = requests.get(authors_contributors, headers=headers)

respons_json = res.json()['data']['children']

for i in range(len(respons_json)):
    print(respons_json[i]['data']['author'])

jazinthapiper
aZestyEggRoll
highfelics
patty3123
Spczippo
Bufflegs69
foobarney
mekkanik
patricknicoconc
mel7it
NeoArdipithecus
SqueakyFarts99
wseQ
PJ-Beans
tabacdk
marccee4
Xeu1009X10
Ca1yp50
kvnsmbrt
Voodoohigh
ryohazuki224
QuasiQuokka
CivitasBlu
plurprincess77
_Jacques


## Example using praw library
- praw library is the python Reddit API wrapper
- We use the same access credentials we used in the previous example

In [34]:
reddit = praw.Reddit(client_id     = CLIENT_ID,
                     client_secret = SECRET_TOKEN,
                     user_agent    = 'MyAPI/0.0.1')

# To test if your instance is working use:
print(reddit.read_only) # Output: True

for submission in reddit.subreddit("learnpython").hot(limit=10):
    print(submission.title)

True
Ask Anything Monday - Weekly Thread
PyStockWatch - a GUI that displays NYSE data.
If you had $3,500 to learn Python, how would you spend it?
Script to watch anime from command line
A question about for loops
If I use requests.get(url) in a try block, would it be reasonable to catch all the exceptions listed here?
Pyperclip module installed but not working?
Although I have created the variable and given it a value it says that it is being referenced before it has been created is there a way to fix this?
How can I be successful with learning Python and the “100 Days of Code” course?
Heroes and Villains


In [35]:
#  getting the wikipages of the r/autowikibot subreddit
#  link to the subreddit http://www.reddit.com/user/autowikibot

for wikipage in reddit.subreddit("autowikibot").wiki:
    print(wikipage)

autowikibot/botlist
autowikibot/commandlist
autowikibot/config/description
autowikibot/config/sidebar
autowikibot/config/stylesheet
autowikibot/config/submit_text
autowikibot/css
autowikibot/excludedsubs
autowikibot/index
autowikibot/livelists
autowikibot/modfaqs
autowikibot/nsfwtag
autowikibot/planned
autowikibot/redditbots
autowikibot/rootonlysubs
autowikibot/statistics
autowikibot/summon
autowikibot/summononlysubs
autowikibot/userblacklist


In [36]:
# getting the content of a specific wikipage in the r/autowikibot subreddit
# link to the wikipage we are requesting https://www.reddit.com/r/autowikibot/wiki/redditbots
wikipage = reddit.subreddit("autowikibot").wiki["redditbots"]

# print the content of the page
# print(wikipage.content_md)

## Using BeautifulSoup

In [37]:
url = "https://www.reddit.com/r/autowikibot/wiki/redditbots"
# Headers to mimic a browser visit
headers = {'User-Agent': 'Mozilla/5.0'}

# Returns a requests.models.Response object
page = requests.get(url, headers=headers)

soup = bs4.BeautifulSoup(page.text, 'html.parser')
a_tags = soup.select("table tbody td a")

bots_usrnames = []
for i in range(len(a_tags)):
    href = a_tags[i].attrs['href']
    if href.startswith('/u/'):
        # print(href)
        bots_usrnames.append(href)
len(bots_usrnames)

399

## Using BeautifulSoup and Pandas

- Pandas provide a `read_html()` function that takes a webpage url and returns a list of dataframes created form the tables that exist in the said webpage.
- I am using here bs4 to parse the webpage and passing the soup object which is the parsed html page I am doing this instead of passing the url to the function because the parser that is integrated with the function is bad.

In [38]:
full_list_bot_usrnames = []

# Parsing a webpage that has one table of bot accounts.
page = requests.get('https://www.reddit.com/r/autowikibot/wiki/redditbots', headers=headers)
soup = bs4.BeautifulSoup(page.text, 'html.parser')

# Passing the parsed page as text to the function
dfs = pd.read_html(page.text)


bots_table = dfs[0]
bots_table.loc[:,'Username']

for username in bots_table.loc[:,'Username']:
    full_list_bot_usrnames.append(username)
    # usrname_url = "https://www.reddit.com" + username
    # print(usrname_url) 
    
df.head()

,downs,score,selftext,subreddit,title,ups,upvote_ratio
0,0.0,6.0,Tell /r/python what you're working on this wee...,Python,Sunday Daily Thread: What's everyone working o...,6.0,0.88
1,0.0,4.0,Have some burning questions on advanced Python...,Python,Tuesday Daily Thread: Advanced questions,4.0,0.84
2,0.0,17.0,Python just makes so much freaking sense! LOVI...,Python,Learning Python,17.0,0.76
3,0.0,79.0,,Python,I'm presenting live in 6 hours at Microsoft Re...,79.0,0.87
4,0.0,551.0,,Python,Python f-strings Are More Powerful Than You Mi...,551.0,0.93


In [39]:
# Parsing a webpage that has two tables of bot accounts.
page = requests.get('https://www.reddit.com/r/botwatch/comments/1wg6f6/bot_list_i_built_a_bot_to_find_other_bots_so_far/cf1nu8p/', headers=headers)
soup = bs4.BeautifulSoup(page.text, 'html.parser')

dfs = pd.read_html(page.text)

# Appending the first dataframe of the firt table to the second dataframe of the second table.
total= dfs[0].append(dfs[1]) 


for username in total.loc[:,'User']:
    # usrname_url = "https://www.reddit.com" + username
    # print(usrname_url) 

    # Checking if the bot account username is already in the list (from the table of bots in the previous list)
    if username not in full_list_bot_usrnames:
        full_list_bot_usrnames.append(username)

In [40]:
print("Number of bot accounts", len(full_list_bot_usrnames))

Number of bot accounts 408
